In [1]:
# python
import sys
import os
import importlib
# columnar analysis
from coffea import processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
from dask.distributed import Client
# local
sys.path.insert(1, os.path.join(sys.path[0], '../..')) # fixme: there must be a better way to handle this...
from sidm.tools import ffschema, sidm_processor, utilities
# always reload local modules to pick up changes during development
importlib.reload(ffschema)
importlib.reload(sidm_processor)
importlib.reload(utilities)
# plotting
import matplotlib.pyplot as plt
utilities.set_plot_style()
%matplotlib inline

In [2]:
samples = [
    "TTJets",
    
    "4mu_1000GeV_0p25GeV_0p002mm",
    "4mu_1000GeV_0p25GeV_0p02mm",
    "4mu_1000GeV_0p25GeV_0p2mm",
    "4mu_1000GeV_0p25GeV_1p0mm",
    "4mu_1000GeV_0p25GeV_2p0mm",

    "4mu_1000GeV_1p2GeV_0p0096mm",
    "4mu_1000GeV_1p2GeV_0p096mm",
    "4mu_1000GeV_1p2GeV_0p96mm",
    "4mu_1000GeV_1p2GeV_4p8mm",
    "4mu_1000GeV_1p2GeV_9p6mm",

    "4mu_1000GeV_5p0GeV_0p04mm",
    "4mu_1000GeV_5p0GeV_0p4mm",
    "4mu_1000GeV_5p0GeV_4p0mm",
    "4mu_1000GeV_5p0GeV_20p0mm",
    "4mu_1000GeV_5p0GeV_40p0mm",

    "4mu_100GeV_0p25GeV_0p02mm",
    "4mu_100GeV_0p25GeV_0p2mm",
    "4mu_100GeV_0p25GeV_2p0mm",
    "4mu_100GeV_0p25GeV_10p0mm",
    "4mu_100GeV_0p25GeV_20p0mm",
    
    "4mu_100GeV_1p2GeV_0p096mm",
    "4mu_100GeV_1p2GeV_0p96mm",
    "4mu_100GeV_1p2GeV_9p6mm",
    "4mu_100GeV_1p2GeV_48p0mm",
    "4mu_100GeV_1p2GeV_96p0mm",
    
    "4mu_100GeV_5p0GeV_0p4mm",
    "4mu_100GeV_5p0GeV_4p0mm",
    "4mu_100GeV_5p0GeV_40p0mm",
    "4mu_100GeV_5p0GeV_200p0mm",
    "4mu_100GeV_5p0GeV_400p0mm",
    
    "4mu_150GeV_0p25GeV_0p013mm",
    "4mu_150GeV_0p25GeV_0p13mm",
    "4mu_150GeV_0p25GeV_1p3mm",
    "4mu_150GeV_0p25GeV_6p7mm",
    "4mu_150GeV_0p25GeV_13p0mm",
    
    "4mu_150GeV_1p2GeV_0p064mm",
    "4mu_150GeV_1p2GeV_0p64mm",
    "4mu_150GeV_1p2GeV_6p4mm",
    "4mu_150GeV_1p2GeV_32p0mm",
    "4mu_150GeV_1p2GeV_64p0mm",
   
    "4mu_150GeV_5p0GeV_0p27mm",
    "4mu_150GeV_5p0GeV_2p7mm",
    "4mu_150GeV_5p0GeV_27p0mm",
    "4mu_150GeV_5p0GeV_130p0mm",
    "4mu_150GeV_5p0GeV_270p0mm",
    
    "4mu_200GeV_0p25GeV_0p01mm",
    "4mu_200GeV_0p25GeV_0p1mm",
    "4mu_200GeV_0p25GeV_1p0mm",
    "4mu_200GeV_0p25GeV_5p0mm",
    "4mu_200GeV_0p25GeV_10p0mm",
    
    "4mu_200GeV_1p2GeV_0p048mm",
    "4mu_200GeV_1p2GeV_0p48mm",
    "4mu_200GeV_1p2GeV_4p8mm",
    "4mu_200GeV_1p2GeV_24p0mm",
    "4mu_200GeV_1p2GeV_48p0mm",
    
    "4mu_200GeV_5p0GeV_0p2mm",
    "4mu_200GeV_5p0GeV_2p0mm",
    "4mu_200GeV_5p0GeV_20p0mm",
    "4mu_200GeV_5p0GeV_100p0mm",
    "4mu_200GeV_5p0GeV_200p0mm",

    "4mu_500GeV_0p25GeV_0p004mm",
    "4mu_500GeV_0p25GeV_0p04mm",
    "4mu_500GeV_0p25GeV_0p4mm",
    "4mu_500GeV_0p25GeV_2p0mm",
    "4mu_500GeV_0p25GeV_4p0mm",
    
    "4mu_500GeV_1p2GeV_0p019mm",
    "4mu_500GeV_1p2GeV_0p19mm",
    "4mu_500GeV_1p2GeV_1p9mm",
    "4mu_500GeV_1p2GeV_9p6mm",
    "4mu_500GeV_1p2GeV_19p0mm",
    
    "4mu_500GeV_5p0GeV_0p08mm",
    "4mu_500GeV_5p0GeV_0p8mm",
    "4mu_500GeV_5p0GeV_8p0mm",
    "4mu_500GeV_5p0GeV_40p0mm",
    "4mu_500GeV_5p0GeV_80p0mm",
    
    "4mu_800GeV_0p25GeV_0p0025mm",
    "4mu_800GeV_0p25GeV_0p025mm",
    "4mu_800GeV_0p25GeV_0p25mm",
    "4mu_800GeV_0p25GeV_1p2mm",
    "4mu_800GeV_0p25GeV_2p5mm",
    
    "4mu_800GeV_1p2GeV_0p012mm",
    "4mu_800GeV_1p2GeV_0p12mm",
    "4mu_800GeV_1p2GeV_1p2mm",
    "4mu_800GeV_1p2GeV_6p0mm",
    "4mu_800GeV_1p2GeV_12p0mm",
    
    "4mu_800GeV_5p0GeV_0p05mm",
    "4mu_800GeV_5p0GeV_0p5mm",
    "4mu_800GeV_5p0GeV_5p0mm",
    "4mu_800GeV_5p0GeV_25p0mm",
    "4mu_800GeV_5p0GeV_50p0mm",
]

In [3]:
samples_4mu = [
#    "TTJets",
    
#    "4mu_100GeV_0p25GeV_0p02mm",
#    "4mu_100GeV_0p25GeV_0p2mm",
#    "4mu_100GeV_0p25GeV_2p0mm",
#    "4mu_100GeV_0p25GeV_10p0mm",
#    "4mu_100GeV_0p25GeV_20p0mm",
    
#    "4mu_100GeV_1p2GeV_0p096mm",
#    "4mu_100GeV_1p2GeV_0p96mm",
#    "4mu_100GeV_1p2GeV_9p6mm",
#    "4mu_100GeV_1p2GeV_48p0mm",
#    "4mu_100GeV_1p2GeV_96p0mm",
    
#    "4mu_100GeV_5p0GeV_0p4mm",
#    "4mu_100GeV_5p0GeV_4p0mm",
#    "4mu_100GeV_5p0GeV_40p0mm",
#    "4mu_100GeV_5p0GeV_200p0mm",
#    "4mu_100GeV_5p0GeV_400p0mm",
    
#    "4mu_150GeV_0p25GeV_0p013mm",
#    "4mu_150GeV_0p25GeV_0p13mm",
#    "4mu_150GeV_0p25GeV_1p3mm",
#    "4mu_150GeV_0p25GeV_6p7mm",
#    "4mu_150GeV_0p25GeV_13p0mm",
    
#    "4mu_150GeV_1p2GeV_0p064mm",
#    "4mu_150GeV_1p2GeV_0p64mm",
#    "4mu_150GeV_1p2GeV_6p4mm",
#    "4mu_150GeV_1p2GeV_32p0mm",
#    "4mu_150GeV_1p2GeV_64p0mm",
   
#    "4mu_150GeV_5p0GeV_0p27mm",
#    "4mu_150GeV_5p0GeV_2p7mm",
#    "4mu_150GeV_5p0GeV_27p0mm",
#    "4mu_150GeV_5p0GeV_130p0mm",
#    "4mu_150GeV_5p0GeV_270p0mm",
    
#    "4mu_200GeV_0p25GeV_0p01mm",
#    "4mu_200GeV_0p25GeV_0p1mm",
#    "4mu_200GeV_0p25GeV_1p0mm",
#    "4mu_200GeV_0p25GeV_5p0mm",
#    "4mu_200GeV_0p25GeV_10p0mm",
    
#    "4mu_200GeV_1p2GeV_0p048mm",
#    "4mu_200GeV_1p2GeV_0p48mm",
#    "4mu_200GeV_1p2GeV_4p8mm",
#    "4mu_200GeV_1p2GeV_24p0mm",
#    "4mu_200GeV_1p2GeV_48p0mm",
    
#    "4mu_200GeV_5p0GeV_0p2mm",
#    "4mu_200GeV_5p0GeV_2p0mm",
#    "4mu_200GeV_5p0GeV_20p0mm",
#    "4mu_200GeV_5p0GeV_100p0mm",
#    "4mu_200GeV_5p0GeV_200p0mm",
    
#    "4mu_500GeV_0p25GeV_0p004mm",
#    "4mu_500GeV_0p25GeV_0p04mm",
#    "4mu_500GeV_0p25GeV_0p4mm",
#    "4mu_500GeV_0p25GeV_2p0mm",
#    "4mu_500GeV_0p25GeV_4p0mm",
    
#    "4mu_500GeV_1p2GeV_0p019mm",
#    "4mu_500GeV_1p2GeV_0p19mm",
#    "4mu_500GeV_1p2GeV_1p9mm",
#    "4mu_500GeV_1p2GeV_9p6mm",
#    "4mu_500GeV_1p2GeV_19p0mm",
    
#    "4mu_500GeV_5p0GeV_0p08mm",
#    "4mu_500GeV_5p0GeV_0p8mm",
#    "4mu_500GeV_5p0GeV_8p0mm",
#    "4mu_500GeV_5p0GeV_40p0mm",
#    "4mu_500GeV_5p0GeV_80p0mm",    
    
#    "4mu_800GeV_0p25GeV_0p0025mm",
#    "4mu_800GeV_0p25GeV_0p025mm",
#    "4mu_800GeV_0p25GeV_0p25mm",
#    "4mu_800GeV_0p25GeV_1p2mm",
#    "4mu_800GeV_0p25GeV_2p5mm",
    
#    "4mu_800GeV_1p2GeV_0p012mm",
#    "4mu_800GeV_1p2GeV_0p12mm",
#    "4mu_800GeV_1p2GeV_1p2mm",
#    "4mu_800GeV_1p2GeV_6p0mm",
#    "4mu_800GeV_1p2GeV_12p0mm",
    
#    "4mu_800GeV_5p0GeV_0p05mm",
#    "4mu_800GeV_5p0GeV_0p5mm",
#    "4mu_800GeV_5p0GeV_5p0mm",
#    "4mu_800GeV_5p0GeV_25p0mm",
#    "4mu_800GeV_5p0GeV_50p0mm",
    
#    "4mu_1000GeV_0p25GeV_0p002mm",
#    "4mu_1000GeV_0p25GeV_0p02mm",
#    "4mu_1000GeV_0p25GeV_0p2mm",
#    "4mu_1000GeV_0p25GeV_1p0mm",
#    "4mu_1000GeV_0p25GeV_2p0mm",

#    "4mu_1000GeV_1p2GeV_0p0096mm",
#    "4mu_1000GeV_1p2GeV_0p096mm",
#    "4mu_1000GeV_1p2GeV_0p96mm",
#    "4mu_1000GeV_1p2GeV_4p8mm",
#    "4mu_1000GeV_1p2GeV_9p6mm",

#    "4mu_1000GeV_5p0GeV_0p04mm",
#    "4mu_1000GeV_5p0GeV_0p4mm",
#    "4mu_1000GeV_5p0GeV_4p0mm",
#    "4mu_1000GeV_5p0GeV_20p0mm",
#    "4mu_1000GeV_5p0GeV_40p0mm",
    
#    '2mu2e_500GeV_5p0GeV_0p08mm',
#    '2mu2e_500GeV_5p0GeV_0p8mm',
#    '2mu2e_500GeV_5p0GeV_8p0mm',
#    '2mu2e_500GeV_5p0GeV_40p0mm',
#    '2mu2e_500GeV_5p0GeV_80p0mm',
]

In [ ]:
fileset = utilities.make_fileset(samples_4mu, "llpNanoAOD_v2")

runner = processor.Runner(
    executor=processor.IterativeExecutor(),
    #executor=processor.FuturesExecutor(),
    schema=NanoAODSchema,
    #maxchunks=1,
)

channels = ["base", "2mu2e", "4mu"]
p = sidm_processor.SidmProcessor(
    channels,
    ["base"],
    lj_reco_choices=["0.4"],
    llpnanoaod=True,
)

output = runner.run(fileset, treename="Events", processor_instance=p)
out = output["out"]

Output()